# EDS Block Testing - Browser Only

This notebook is designed for **browser execution only** using the ipynb-viewer block.

## How to Use

1. **View on EDS site**: Add the ipynb-viewer block pointing to this notebook
2. **Run Cell 1 first**: Initializes the browser environment
3. **Run any cell**: Click "Run" buttons to execute code cells

## Important Notes

- **Browser-only execution**: Runs in your browser via ipynb-viewer block
- **Always run Cell 1 first**: Sets up helper functions on window object
- **Interactive previews**: Opens popup windows with live block decoration
- **Native browser APIs**: Uses document, window directly - no virtual DOM

In [ ]:
// ============================================================================
// SETUP: Initialize browser environment
// ============================================================================
// Run this cell first! It sets up all helper functions.

(async () => {
  const { initialize } = await import('/scripts/ipynb-helpers.js');
  await initialize();
  return '✅ Browser environment ready';
})();

## Part 1: Simple DOM Test

Test basic DOM functionality using native browser APIs.

In [ ]:
// Create a simple DOM element
(async () => {
  const testDiv = document.createElement('div');
  testDiv.textContent = 'Hello from Browser!';
  testDiv.className = 'test';
  testDiv.style.cssText = 'padding: 20px; background: #e3f2fd; border-radius: 8px; color: #1976d2; font-weight: bold;';
  
  console.log('✓ Created element:', testDiv.outerHTML);
  
  return testDiv.outerHTML;
})();

## Part 2: Testing Blocks

Test EDS blocks with the helper functions provided by Cell 1.

In [ ]:
// Test HelloWorld block (no content needed)
(async () => {
  const helloBlock = await window.testBlockFn('helloworld');
  console.log('✓ HelloWorld block created');
  
  return helloBlock.outerHTML;
})();

In [ ]:
// Test Accordion block - BEFORE and AFTER
(async () => {
  const accordionContent = `
    <div>
      <div>What is EDS?</div>
      <div>Edge Delivery Services is Adobe's modern web platform.</div>
    </div>
    <div>
      <div>How do blocks work?</div>
      <div>Blocks transform DOM elements using JavaScript decoration.</div>
    </div>
  `;
  
  // BEFORE
  const before = document.createElement('div');
  before.className = 'accordion';
  before.innerHTML = accordionContent;
  const beforeHTML = before.innerHTML.trim().substring(0, 100);
  
  // AFTER
  const after = await window.testBlockFn('accordion', accordionContent);
  const afterHTML = after.innerHTML.trim().substring(0, 100);
  
  return `BEFORE:\n${beforeHTML}...\n\nAFTER:\n${afterHTML}...\n\nTransformed: ${before.children.length} divs → ${after.querySelectorAll('details').length} details`;
})();

## Part 3: Visual Preview with Popup Window

Create visual previews that open in a popup window with full styling!

**Note**: Make sure popups are allowed for this domain.

In [ ]:
// Create visual preview in popup window
(async () => {
  const accordionContent = `
    <div>
      <div>What is EDS?</div>
      <div>Edge Delivery Services is Adobe's modern web platform for building fast, performant websites.</div>
    </div>
    <div>
      <div>How do blocks work?</div>
      <div>Blocks are JavaScript functions that decorate DOM elements and transform content structure.</div>
    </div>
    <div>
      <div>Why use ipynb-viewer?</div>
      <div>The ipynb-viewer block lets end users interact with executable notebooks directly on your EDS site.</div>
    </div>
  `;
  
  // Opens popup window with styled block!
  await window.showPreview('accordion', accordionContent);
  
  return '✓ Preview window opened (check for popup!)';
})();

## Part 4: Available Blocks

List of common blocks you can test in this project.

In [ ]:
// Common blocks available for testing
(async () => {
  const commonBlocks = [
    'accordion',
    'cards',
    'columns',
    'fragment',
    'header',
    'footer',
    'hero',
    'helloworld',
    'ipynb-viewer'
  ];
  
  console.log('Available blocks:', commonBlocks.join(', '));
  
  return commonBlocks;
})();

## Part 5: Interactive Calculations

Pure JavaScript that works in the browser.

In [ ]:
// Simple calculation
const a = 10;
const b = 20;
const sum = a + b;

console.log('Sum:', sum);
console.log('Product:', a * b);
console.log('Average:', sum / 2);

sum

In [ ]:
// Array operations
const numbers = [1, 2, 3, 4, 5];
const doubled = numbers.map(n => n * 2);
const sumNums = numbers.reduce((acc, n) => acc + n, 0);

console.log('Original:', numbers);
console.log('Doubled:', doubled);
console.log('Sum:', sumNums);

doubled

## Part 6: String Manipulation

In [ ]:
// Text transformations
const text = 'hello world from jupyter';
const upper = text.toUpperCase();
const titleCase = text.replace(/\b\w/g, c => c.toUpperCase());

console.log('Original:', text);
console.log('Uppercase:', upper);
console.log('Title Case:', titleCase);

titleCase

## Quick Reference

### Helper Functions (Available After Cell 1)

After running Cell 1, these functions are available on the window object:

**Test blocks:**
```javascript
const block = await window.testBlockFn('blockname', '<div>content</div>');
```

**Show preview (opens popup):**
```javascript
await window.showPreview('blockname', '<div>content</div>');
```

**Access document:**
```javascript
const div = window.doc.createElement('div');
// or just use document directly
const div = document.createElement('div');
```

### Code Pattern

Always wrap async code in an IIFE:

```javascript
(async () => {
  const block = await window.testBlockFn('blockname', '<div>content</div>');
  await window.showPreview('blockname', '<div>content</div>');
  return block.outerHTML;
})();
```

### Preview Features

- Opens in popup window
- Full CSS styling via base tag
- Block JavaScript decoration
- Interactive elements work
- Close button or press ESC
- Refresh button to reload

### Tips

✅ Always run Cell 1 first

✅ Wrap async code in IIFE: `(async () => { ... })()`

✅ Use `window.testBlockFn` and `window.showPreview`

✅ Check console.log for debugging

✅ Allow popups for your domain to see previews

❌ Don't forget to run Cell 1 before other cells